# Studio Notebooks と Training Jobs のコンテナを統一する
機械学習のコード開発において、開発環境と実行環境に差異があると、動かない可能性がある。  
このノートブックでは開発環境として Studio Notebooks, 実行環境として Traning Jobs を使用することとし、同一環境（コンテナイメージ）で実行するための手順を紹介。

## 前提条件
* SageMaker Studio のドメインが出来上がっている
    * 使用する SageMaker Studio の DomainId をメモしてください。後ほど使用します。
* このノートブックを SageMaker Notebooks (≠Studio)で実行する
    * docker コマンドを使用できて、SageMakerFullAccess 相当の権限があれば他の環境でも実行できますが、その場合は role などを適切に設定してください
    * docker コマンドが正常に機能しないため、SageMaker Studio Notebooks では動かせません。このノートブックはあくまで環境を作成(コンテナイメージ作成〜SageMaker STudio への登録）するものであり、Studio で動かすノートブックは、このノートブックを動かしたあとに [2_studio.ipynb](./2_studio.ipynb) を実行します。

## 使用するモジュールや定数の設定
[constant_config.yml](./constant_config.yml) に定数が設定されているので必要に応じて変更する。

In [ ]:
import yaml
import boto3
import sagemaker
from time import sleep
region = boto3.session.Session().region_name
account = boto3.client('sts').get_caller_identity().get('Account')
ecr = boto3.client('ecr')
sm_client = boto3.client('sagemaker')


# 定数ファイル読み込み
with open('constant_config.yml','rt') as f:
    constant_config = yaml.safe_load(f)

REPOSITORY_NAME = constant_config['REPOSITORY_NAME']
IMAGE_NAME = constant_config['IMAGE_NAME']
KERNEL_NAME = constant_config['KERNEL_NAME']
DISPLAY_NAME = constant_config['DISPLAY_NAME']
role = sagemaker.get_execution_role()
image_uri = f'{account}.dkr.ecr.{region}.amazonaws.com/{REPOSITORY_NAME}:{IMAGE_NAME}'
print(f'region: {region}')
print(f'account : {account}')
print(f'role : {role}')
print(f'REPOSITORY_NAME : {REPOSITORY_NAME}')
print(f'IMAGE_NAME : {IMAGE_NAME}')
print(f'KERNEL_NAME : {KERNEL_NAME}')
print(f'DISPLAY_NAME : {DISPLAY_NAME}')
print(f'image_uri : {image_uri}')

## コンテナイメージの準備
### コンテナイメージの作成と登録
Studio Notebooks も Tranining Jobs もコンテナで動くことから、まずはコンテナイメージを build し、ECR に push する。
#### ECR のリポジトリ作成

In [ ]:
!aws ecr create-repository --repository-name {REPOSITORY_NAME}

#### コンテナイメージにインストールするモジュールの列挙
使用するものを requirements.txt に書き込む。  
機械学習で使用するモジュール(今回は `numpy` ) とは別に、Studio Notebooks で使用するために、`ipykernel` を、Training Jobs で使用するために `sagemaker-training` をインストールする。

In [ ]:
%%writefile requirements.txt
ipykernel==6.20.1
numpy==1.22.3
sagemaker==2.144.0
sagemaker-training==4.4.8

#### Dockerfile の作成
ポイントは、Studio(Jupyter) で使うために、`ipykernel install` しているところ。  
また、使用者がどんな風にコンテナを作ったかがわかるように（追加モジュールのリクエストやバージョンの確認のため）、 `requirements.txt` と `Dockerfile` も同包している。

In [ ]:
%%writefile Dockerfile
FROM python:3.10.10-buster

COPY requirements.txt /root/requirements.txt
COPY Dockerfile /root/Dockerfile

RUN pip install -r /root/requirements.txt && \
        python -m ipykernel install --sys-prefix

####  コンテナイメージのビルド

In [ ]:
!docker build . -t {IMAGE_NAME} -t {image_uri}

#### コンテナイメージのプッシュ

In [ ]:
!$(aws ecr get-login --region {region} --registry-ids {account} --no-include-email)
!docker push {image_uri}

### コンテナイメージを SageMaker に登録する
手順としては、  
1. SageMaker 側のコンテナイメージの箱であるイメージを作成(`create_image`)する
2. そこに ECR のイメージを登録(`create_image_version`)する。
3. イメージの運用設定(uid, gid, mountpoint, Studio での表示名など)を登録(`create_app_image_config`)する
4. SageMaker Studio のドメインに紐づけ(update_domain)て使用可能にする

#### create_image

In [ ]:
sm_client.create_image(
    ImageName=IMAGE_NAME,
    RoleArn=role,
)

#### create_image_version

In [ ]:
sleep(1) # create_image は非同期 API のため、wait を入れる
sm_client.create_image_version(
    BaseImage=image_uri,
    ImageName=IMAGE_NAME,
)

In [ ]:
# バージョンが登録されたかチェック(ImageVersionStatus)
sm_client.describe_image_version(
    ImageName=IMAGE_NAME
)

#### create_app_image_config
事前にコンテナのデフォルトの uid, gid を確認する

In [ ]:
output = !docker run {image_uri} id -u
default_uid = int(output[0])
output = !docker run {image_uri} id -g
default_gid = int(output[0])
print(default_uid, default_gid)

In [ ]:
sm_client.create_app_image_config(
    AppImageConfigName=IMAGE_NAME,
    KernelGatewayImageConfig={
        'KernelSpecs': [
            {
                'Name': KERNEL_NAME,
                'DisplayName': DISPLAY_NAME
            },
        ],
        'FileSystemConfig': {
            'MountPath': '/root/data',
            'DefaultUid': default_uid,
            'DefaultGid': default_gid
        }
    }
)

#### update_domain
ここでは list_domains()で取得する先頭の domain に紐付ける。
本来は以下の domain_id に紐付けたい domain を設定する。

In [ ]:
# 例えば
domain_id = sm_client.list_domains()['Domains'][0]['DomainId']
# 本来は
# domain_id = 'type your domain id'

In [ ]:
sm_client.update_domain(
    DomainId = domain_id,
    DefaultUserSettings = {
        'KernelGatewayAppSettings' : {
            'CustomImages' : [
                {
                    'ImageName' : IMAGE_NAME,
                    'AppImageConfigName' : IMAGE_NAME
                }
            ]
        }
    }
)

In [ ]:
# check
sm_client.describe_domain(
    DomainId = domain_id
)

以降は SageMaker Studio で、改めて  
`git clone https://github.com/kazuhitogo/sagemaker-studio-byoc`  
したあと、[2_studio.ipynb](2_studio.ipynb) を実行してください。  
また、SageMaker Studio を立ち上げる際は、UserProfile から、Default カーネルを事前に削除してから Studio を開いてください。
![delete_default](./img/delete_default.png)
2_studio.ipynb が完了したら以降のお片付けを実行してください。

## お片付け

In [ ]:
sm_client.update_domain(
    DomainId = domain_id,
    DefaultUserSettings = {
        'KernelGatewayAppSettings' : {
            'CustomImages' : []
        }
    }
)

In [ ]:
sm_client.delete_app_image_config(
    AppImageConfigName = IMAGE_NAME
)

In [ ]:
sm_client.delete_image(
    ImageName=IMAGE_NAME,
)

In [ ]:
!aws ecr delete-repository --repository-name {REPOSITORY_NAME} --force